<a href="https://colab.research.google.com/github/kuonumber/crawler/blob/master/Day035_muti_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
import threading
import time

maxthreads = 2
sema = threading.Semaphore(value=maxthreads)
threads = list()

def task(i):
    sema.acquire()
    print( "start %s" % (i,))
    time.sleep(2)
    sema.release()

for i in range(10):
    thread = threading.Thread(target=task,args=(i,))
    threads.append(thread)
    thread.start()

print('Main thread done')


start 0
start 1
Main thread done


In [ ]:

maxthreads = 2
sema = threading.Semaphore(value=maxthreads)
threads = list()

def task(i):
    sema.acquire()
    print( "start %s" % (i,))
    time.sleep(2)
    sema.release()

for i in range(10):
    thread = threading.Thread(target=task,args=(i,))
    threads.append(thread)
    thread.start()

for t in threads:
    t.join()

print('Main thread done')


start 0
start 1
start 2
start 3
start 4
start 5
start 6
start 7
start 8
start 9
Main thread done


In [ ]:
import threading
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [ ]:
%%timeit

datetime.now()
# 子執行緒的工作函數
def job():
    max_page = 50
    for page_number in range(1, max_page+1, 1):
        url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
        payload = {'page':str(page_number)}
        resp = requests.get(url, params=payload)
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.text, 'lxml')

        # 把電影介紹擷取出來
        movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
        for p in movie_list:
            print("----------------------------------------------------------------------")
            # 電影名稱
            movie_name = p.find("div", attrs={"class":"release_movie_name"})
            print("電影名稱：", movie_name.a.string)
            
            # 定位電影評價資訊欄位
            level_box = movie_name.find("dl", attrs={"class":"levelbox"})
            
            # 期待度
            expectation = level_box.findAll("div", attrs={"class":"leveltext"})[0]
            print("期待度：", expectation.span.string)
            
            # 滿意度
            satisfaction = level_box.findAll("div", attrs={"class":"leveltext"})[1]
            print("滿意度：", satisfaction.span["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
            
            # 簡介
            movie_info = p.find("div", attrs={"class":"release_text"})
            print(movie_info.span.string)

# 建立一個子執行緒
t = threading.Thread(target = job)

# 執行該子執行緒
t.start()

# 主執行緒繼續執行自己的工作
for i in range(10):
  print("Main thread:", i)
#   time.sleep(1)

# 等待 t 這個子執行緒結束
t.join()

print("Done.")

In [ ]:
%%timeit

maxthreads = 2
sema = threading.Semaphore(value=maxthreads)
threads = list()

def task(i):
    sema.acquire()
    print( "start %s" % (i,))
    time.sleep(2)
    sema.release()

def task(page_number):
    sema.acquire()
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')
    print( "start %s" % (page_number,))

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("----------------------------------------------------------------------")
        # 電影名稱
        movie_name = p.find("div", attrs={"class":"release_movie_name"})
        print("電影名稱：", movie_name.a.string)
        
        # 定位電影評價資訊欄位
        level_box = movie_name.find("dl", attrs={"class":"levelbox"})
        
        # 期待度
        expectation = level_box.findAll("div", attrs={"class":"leveltext"})[0]
        print("期待度：", expectation.span.string)
        
        # 滿意度
        satisfaction = level_box.findAll("div", attrs={"class":"leveltext"})[1]
        print("滿意度：", satisfaction.span["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
        
        # 簡介
        movie_info = p.find("div", attrs={"class":"release_text"})
        print(movie_info.span.string)   
    sema.release()

max_page = 10
for page_number in range(1, max_page+1, 1):
    thread = threading.Thread(target=task,args=(page_number,))
    threads.append(thread)
    thread.start()

for t in threads:
    t.join()

print('Main thread done')


In [ ]:
%%timeit

max_page = 10
for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("----------------------------------------------------------------------")
        # 電影名稱
        movie_name = p.find("div", attrs={"class":"release_movie_name"})
        print("電影名稱：", movie_name.a.string)
        
        # 定位電影評價資訊欄位
        level_box = movie_name.find("dl", attrs={"class":"levelbox"})
        
        # 期待度
        expectation = level_box.findAll("div", attrs={"class":"leveltext"})[0]
        print("期待度：", expectation.span.string)
        
        # 滿意度
        satisfaction = level_box.findAll("div", attrs={"class":"leveltext"})[1]
        print("滿意度：", satisfaction.span["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
        
        # 簡介
        movie_info = p.find("div", attrs={"class":"release_text"})
        print(movie_info.span.string)